## 3D-GAN

In [ ]:
import numpy as np
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
import os
import time

### Configuration

In [ ]:
tf.reset_default_graph()
np.random.seed(1) 
tf.set_random_seed(1)

training_epochs = 20001 # num of epochs to train discriminator and generator jointly
batch_size = 64 # batch size for training the model

# ---- max layer 256 for generator ---#
#alpha_g = 0.00033955 # generator's learning rate
#alpha_d = 0.0000198 # discriminator's learning rate

#alpha_g = 0.0004
#alpha_d = 0.000018

# okay until 4k epochs
#alpha_g = 0.001
#alpha_d = 0.000018

# ---- max layer 512 for generator ---#
# okay until 3k epochs
#alpha_g = 0.0015
#alpha_d = 0.00002

alpha_g = 0.0015
alpha_d = 0.000024

beta1 = 0.5 # the fraction factor used in the first momentum term from Adam optimizer
logs_path = "./3d_gan_log" # directory to save the training log to
train_sample_directory = './3d_gan/train_sample/' # directory to save the generated 3d model during training
model_directory = './3d_gan/models' # directory to save trained model

### Read training 3D arrays

In [ ]:
train_dir = './3D_Voxel_Files/'
train_names = [f for f in os.listdir(train_dir) if f.endswith('_1')]

chairs = []
for f in train_names:
    chairs.append(np.load(train_dir+f)) # load the binary array files
train_chairs = np.array(chairs)
train_chairs=train_chairs.reshape([988,32,32,32,1]) # turn train_chairs into 5D tensor [batch, depth, height, width, channels]

In [ ]:
train_chairs.shape

### Declare helpful functions

In [ ]:
# get a batch of real 3D models
def get_x(batch_size):
    indices = np.random.randint(len(train_chairs), size=batch_size) # random sample real images
    batch = train_chairs[indices] 
    return batch

## Define the Generative Adversarial Network

#### Generator network

In [ ]:
# when is_train=False, do not perform batch normalization
def generator(z, batch_size=batch_size, is_train=True, reuse=False):
    print '\n Generator:'
    
    if reuse:
        tf.get_variable_scope().reuse_variables()
        tl.layers.set_name_reuse(reuse)
    
    # input of the network 
    gen_in = tl.layers.InputLayer(inputs=z, name='g/in')
    # fully connected layer
    # n_units: number of nodes in FC layer
    gen_h0 = tl.layers.DenseLayer(layer=gen_in, n_units = 4*4*4*512,
                                W_init = initializer,
                                act = tf.identity, name='g/h0/lin')
    #print '!!!gen_h0.outputs before reshape:',gen_h0.outputs
    # reshape the fully connected layer to a 5D tensor [batch size, depth, height, width, channels]
    # output size: batch_size*4*4*4*512
    gen_h0 = tl.layers.ReshapeLayer(gen_h0, shape = [-1,4,4,4,512], name='g/h0/reshape')
    # batch normalize weight parameters
    gen_h0 = tl.layers.BatchNormLayer(gen_h0, is_train=is_train, gamma_init=tf.random_normal_initializer(1., 0.02), name='g/h0/batch_norm')
    # apply activation function
    gen_h0.outputs = tf.nn.relu(gen_h0.outputs, name='g/h0/relu')
    #print '!!!gen_h0.outputs after reshape:',gen_h0.outputs 
    
    # shape: [kernel_depth, kernel_height, kernel_width, output_channels, input_channels]
    # output shape: [batch size, output_depth, output_height, output_width, output channels]
    # strides: [stride on batch, depth, height, width, channels]
    # act: activation function
    # output size: batch_size*8*8*8*256
    gen_h1 = tl.layers.DeConv3dLayer(layer=gen_h0,
                                shape = [4,4,4,256,512],
                                output_shape = [batch_size,8,8,8,256],
                                strides=[1,2,2,2,1],
                                W_init = initializer,
                                act=tf.identity, name='g/h1/decon2d')
    gen_h1 = tl.layers.BatchNormLayer(gen_h1, is_train=is_train, gamma_init=tf.random_normal_initializer(1., 0.02), name='g/h1/batch_norm')
    gen_h1.outputs = tf.nn.relu(gen_h1.outputs, name='g/h1/relu')
    #print '!!!gen_h1.outputs:',gen_h1.outputs

    # output size: batch_size*16*16*16*128
    gen_h2 = tl.layers.DeConv3dLayer(layer=gen_h1,
                                shape = [4,4,4,128,256],
                                output_shape = [batch_size,16,16,16,128],
                                strides=[1,2,2,2,1],
                                W_init = initializer,
                                act=tf.identity, name='g/h2/decon2d')
    gen_h2 = tl.layers.BatchNormLayer(gen_h2, is_train=is_train, gamma_init=tf.random_normal_initializer(1., 0.02), name='g/h2/batch_norm')
    gen_h2.outputs = tf.nn.relu(gen_h2.outputs, name='g/h2/relu')
    #print '!!!gen_h2.outputs:',gen_h2.outputs 
    
    # output size: batch_size*32*32*32*1
    gen_h3 = tl.layers.DeConv3dLayer(layer=gen_h2,
                                shape = [4,4,4,1,128],
                                output_shape = [batch_size,32,32,32,1],
                                strides=[1,2,2,2,1],
                                W_init = initializer,
                                act=tf.identity, name='g/h3/decon2d')
    gen_h3.outputs = tf.nn.sigmoid(gen_h3.outputs)
    #print '!!!gen_h3.outputs:',gen_h3.outputs
    
    return gen_h3 # return the generator network

In [ ]:
# when is_train=False, do not perform batch normalization
def generator1(z, batch_size=batch_size, is_train=True, reuse=False):
    print '\n Generator:'
    
    if reuse:
        tf.get_variable_scope().reuse_variables()
        tl.layers.set_name_reuse(reuse)
    
    # input of the network 
    gen_in = tl.layers.InputLayer(inputs=z, name='g/in')
    # fully connected layer
    # n_units: number of nodes in FC layer
    gen_h0 = tl.layers.DenseLayer(layer=gen_in, n_units = 4*4*4*256,
                                W_init = initializer,
                                act = tf.identity, name='g/h0/lin')
    #print '!!!gen_h0.outputs before reshape:',gen_h0.outputs
    # reshape the fully connected layer to a 5D tensor [batch size, depth, height, width, channels]
    # output size: batch_size*4*4*4*256
    gen_h0 = tl.layers.ReshapeLayer(gen_h0, shape = [-1,4,4,4,256], name='g/h0/reshape')
    # batch normalize weight parameters
    gen_h0 = tl.layers.BatchNormLayer(gen_h0, is_train=is_train, gamma_init=tf.random_normal_initializer(1., 0.02), name='g/h0/batch_norm')
    # apply activation function
    gen_h0.outputs = tf.nn.relu(gen_h0.outputs, name='g/h0/relu')
    #print '!!!gen_h0.outputs after reshape:',gen_h0.outputs 
    
    # shape: [kernel_depth, kernel_height, kernel_width, output_channels, input_channels]
    # output shape: [batch size, output_depth, output_height, output_width, output channels]
    # strides: [stride on batch, depth, height, width, channels]
    # act: activation function
    # output size: batch_size*8*8*8*128
    gen_h1 = tl.layers.DeConv3dLayer(layer=gen_h0,
                                shape = [4,4,4,128,256],
                                output_shape = [batch_size,8,8,8,128],
                                strides=[1,2,2,2,1],
                                W_init = initializer,
                                act=tf.identity, name='g/h1/decon2d')
    gen_h1 = tl.layers.BatchNormLayer(gen_h1, is_train=is_train, gamma_init=tf.random_normal_initializer(1., 0.02), name='g/h1/batch_norm')
    gen_h1.outputs = tf.nn.relu(gen_h1.outputs, name='g/h1/relu')
    #print '!!!gen_h1.outputs:',gen_h1.outputs

    # output size: batch_size*16*16*16*64
    gen_h2 = tl.layers.DeConv3dLayer(layer=gen_h1,
                                shape = [4,4,4,64,128],
                                output_shape = [batch_size,16,16,16,64],
                                strides=[1,2,2,2,1],
                                W_init = initializer,
                                act=tf.identity, name='g/h2/decon2d')
    gen_h2 = tl.layers.BatchNormLayer(gen_h2, is_train=is_train, gamma_init=tf.random_normal_initializer(1., 0.02), name='g/h2/batch_norm')
    gen_h2.outputs = tf.nn.relu(gen_h2.outputs, name='g/h2/relu')
    #print '!!!gen_h2.outputs:',gen_h2.outputs 
    
    # output size: batch_size*32*32*32*1
    gen_h3 = tl.layers.DeConv3dLayer(layer=gen_h2,
                                shape = [4,4,4,1,64],
                                output_shape = [batch_size,32,32,32,1],
                                strides=[1,2,2,2,1],
                                W_init = initializer,
                                act=tf.identity, name='g/h3/decon2d')
    gen_h3.outputs = tf.nn.sigmoid(gen_h3.outputs)
    #print '!!!gen_h3.outputs:',gen_h3.outputs
    
    return gen_h3 # return the generator network

#### Discriminator network

In [ ]:
# when is_train=False, do not perform batch normalization
def discriminator(inputs, batch_size=batch_size, is_train=True, reuse=False):
    print '\n Discriminator:'
    
    if reuse:
        tf.get_variable_scope().reuse_variables()
        tl.layers.set_name_reuse(reuse)
    
    # Inputs size: batch_size*32*32*32*1 
    dis_in = tl.layers.InputLayer(inputs, name='d/in')
    #print '!!!dis_in:',
    #dis_in.print_layers() 
    
    # Creates 32 4*4*4 filters to convolve on the mini-batch of 32*32*32 chairs, also perform batch norm + leaky ReLU activation
    # Set reuse=True allows discriminator to evaluate both real samples and generated samples 
    # shape: [kernel_depth, kernel_height, kernel_width, input_channels, output_channels]
    # strides: [stride on batch, depth, height, width, channels]
    # act: activation function
    # Outputs size: batch_size*16*16*16*32 
    dis_h0 = tl.layers.Conv3dLayer(dis_in, shape=[4,4,4,1,32],
                                   W_init = initializer,
                                   strides=[1,2,2,2,1], name='d/h0/conv2d')
    # alpha: degree of leakiness used in leaky ReLU
    dis_h0.outputs = tl.activation.leaky_relu(dis_h0.outputs, alpha=0.2, name='d/h0/lrelu')
    #print '!!!dis_h0.outputs:',dis_h0.outputs 
    
    # outputs size: batch_size*8*8*8*64 
    dis_h1 = tl.layers.Conv3dLayer(dis_h0, shape=[4,4,4,32,64],
                                   W_init = initializer,
                                   strides=[1,2,2,2,1], name='d/h1/conv2d')
    dis_h1 = tl.layers.BatchNormLayer(dis_h1, is_train=is_train, name='d/h1/batch_norm')
    dis_h1.outputs = tl.activation.leaky_relu(dis_h1.outputs, alpha=0.2, name='d/h1/lrelu')
    #print '!!!dis_h1.outputs:',dis_h1.outputs  
    
    # outputs size batch_size*4*4*4*128 
    dis_h2 = tl.layers.Conv3dLayer(dis_h1, shape=[4,4,4,64,128],
                                   W_init = initializer,
                                   strides=[1,2,2,2,1], name='d/h2/conv2d')
    dis_h2 = tl.layers.BatchNormLayer(dis_h2, is_train=is_train, name='d/h2/batch_norm')
    dis_h2.outputs = tl.activation.leaky_relu(dis_h2.outputs, alpha=0.2, name='d/h2/lrelu')
    #print '!!!dis_h2.outputs:',dis_h2.outputs   
    
    # outputs size batch_size*2*2*2*256 
    dis_h3 = tl.layers.Conv3dLayer(dis_h2, shape=[4,4,4,128,256],
                                   W_init = initializer,
                                   strides=[1,2,2,2,1], name='d/h3/conv2d')
    dis_h3 = tl.layers.BatchNormLayer(dis_h3, is_train=is_train, name='d/h3/batch_norm')
    dis_h3.outputs = tl.activation.leaky_relu(dis_h3.outputs, alpha=0.2, name='d/h3/lrelu')
    #print '!!!dis_h3.outputs:',dis_h3.outputs  
    
    # !!!!!! I have edited /Library/Python/2.7/site-packages/tensorlayer/layers.py to output dimension 2*2*2*256
    # !!!!!! Remember to change this to according values when the discriminator's structure changed
    #print '!!!',
    #dis_h3.print_layers() # print all information about dis_h3
    
    # flatten the tensor to [batch_size, 2*2*2*256]
    dis_flat = tl.layers.FlattenLayer(dis_h3, name='d/h4/flatten')
    
    # create a fully connect layer with dis_flat and just one node in the output layer
    # note there's no batch normalization at this layer
    # n_units: number of nodes in FC layer
    dis_output = tl.layers.DenseLayer(dis_flat, n_units=1, act=tf.identity,
                                    W_init=initializer,
                                    name='d/h4/lin_sigmoid')
    # outputs size: batch_size*1 
    logits = dis_output.outputs
    dis_output.outputs = tf.nn.sigmoid(dis_output.outputs)    
    #print '!!!dis_output.outputs:',dis_output.outputs 
    #print '!!!logits:',logits 
    
    return dis_output, logits

### Connecting two networks together

In [ ]:
z_size = 100 # size of initial noise vector that will be used for generator

# initialize all parameters of the networks
# weights were initialized from a zero-centered Normal distribution with standard deviation 0.02
# tf.truncated_normal returns random values from a normal distribution and made sure no value exceeds 2 std
initializer = tf.truncated_normal_initializer(stddev=0.02)

# placeholders for inputs into the generator and discriminator, respectively.
z_vector = tf.placeholder(shape=[batch_size,z_size],dtype=tf.float32, name='z_vectors') 
x_vector = tf.placeholder(shape=[batch_size,32,32,32,1],dtype=tf.float32, name='real_chairs') 


# ---- DCGAN ----
net_g_train = generator(z_vector, is_train=True, reuse=False) # generated mini-batch of images from noisy z vectors 
print 'generator: ',net_g_train
#print 'generator outputs: ',net_g_train.outputs

net_d, d_output_x = discriminator(x_vector, is_train=True, reuse=False) # probabilities for real images                !!! should have ,reuse=True when there's pre-training
print 'discriminator: ',net_d
d_output_x = tf.maximum(tf.minimum(d_output_x, 0.99), 0.01) # avoid inf and -inf
summary_d_x_hist = tf.histogram_summary("d_prob_x", d_output_x)

net_d2, d_output_z = discriminator(net_g_train.outputs, is_train=True, reuse=True) # probabilities for generated images
d_output_z = tf.maximum(tf.minimum(d_output_z, 0.99), 0.01) # avoid inf and -inf
summary_d_z_hist = tf.histogram_summary("d_prob_z", d_output_z)

d_loss = -tf.reduce_mean(tf.log(d_output_x) + tf.log(1-d_output_z)) # loss for discriminator
summary_d_loss = tf.scalar_summary("d_loss", d_loss)

g_loss = -tf.reduce_mean(tf.log(d_output_z)) # loss for generator
summary_g_loss = tf.scalar_summary("g_loss", g_loss)

# this is generator for evaluation, set is_train to False so that BatchNormLayer behave differently
net_g_test = generator(z_vector, is_train=False, reuse=True)


# the following parameter indices may change if the network structure changes
para_g=list(np.array(tf.trainable_variables())[[0,1,4,5,8,9,12,13]])
para_d=list(np.array(tf.trainable_variables())[[14,15,16,17,20,21,24,25,28,29]])

# only update parameters in discriminator during pre-training
#pre_optimizer = tf.train.AdamOptimizer(learning_rate=alpha_d,beta1=beta1).minimize(d_pre_loss,var_list=para_d)
# only update the weights for the discriminator network
optimizer_op_d = tf.train.AdamOptimizer(learning_rate=alpha_d,beta1=beta1).minimize(d_loss,var_list=para_d)
# only update the weights for the generator network
optimizer_op_g = tf.train.AdamOptimizer(learning_rate=alpha_g,beta1=beta1).minimize(g_loss,var_list=para_g)

### Train the GAN model

In [ ]:
# create a log folder and save the graph structure, do this before training
g_writer = tf.train.SummaryWriter(logs_path + '/generator', graph=tf.get_default_graph())
d_writer = tf.train.SummaryWriter(logs_path + '/discriminator')

# saver saves and loads variables of the model to and from checkpoints, 
# which are binary files that maps variable names to tensor values
saver = tf.train.Saver(max_to_keep=50) 

with tf.Session() as sess:  
    # set GPU memeory fraction
    tl.ops.set_gpu_fraction(sess=sess, gpu_fraction=0.99)
    
    # variables need to be initialized before we can use them
    sess.run(tf.initialize_all_variables())
    #print [v.name for v in tf.trainable_variables()] # print all variable names
    #print 'para_g:',[v.name for v in para_g]
    #print '\n para_d:',[v.name for v in para_d]
    
    
    # -------- jointly training discriminator and generator --------
    start = time.time()
    
    # z noise vector that will be used to generate chairs to check the training progress
    #z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) # uniform distribution between [-1, 1]
    z_sample = np.random.normal(0, 0.33, size=[batch_size,z_size]).astype(np.float32) # gaussian distribution between [-1, 1]
    
    for epoch in range(training_epochs):
        # get a batch of real chairs, with range [-1 ,1]
        x = get_x(batch_size) 
        # mini-batch of noise data from [-1, 1]
        #z = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
        z = np.random.normal(0, 0.33, size=[batch_size,z_size]).astype(np.float32)
    
        # Update the discriminator and generator
        d_summary_merge = tf.merge_summary([summary_d_loss, summary_d_x_hist,summary_d_z_hist])
        summary_d,discriminator_loss = sess.run([d_summary_merge,d_loss],feed_dict={z_vector:z, x_vector:x})
        summary_g,genterator_loss = sess.run([summary_g_loss,g_loss],feed_dict={z_vector:z})  
        #print "epoch: ", epoch
        #print 'd_loss:',discriminator_loss
        #print 'g_loss:',genterator_loss
        
        # only update the discriminator when its loss is larger than 10%       !!!!! D may not get sufficiently trained if threshold is too high
        if discriminator_loss <= 4.6*0.1: 
            sess.run([optimizer_op_g],feed_dict={z_vector:z})
            #print "epoch: ",epoch,', d_loss:',discriminator_loss,'g_loss:',genterator_loss
        # only update the generator when its loss is larger than 10%       !!!!! G may not get sufficiently trained if threshold is too high
        elif genterator_loss <= 4.6*0.1:
            sess.run([optimizer_op_d],feed_dict={z_vector:z, x_vector:x})
            #print "epoch: ",epoch,', d_loss:',discriminator_loss,'g_loss:',genterator_loss
        else:
            sess.run([optimizer_op_d],feed_dict={z_vector:z, x_vector:x})
            sess.run([optimizer_op_g],feed_dict={z_vector:z})
        
        # add loss summary to tensorboard
        if epoch % 5 == 0:
            d_writer.add_summary(summary_d, epoch) 
            g_writer.add_summary(summary_g, epoch)
        
        # print training progress
        if epoch % 100 == 0:
            time_lapse = time.time()-start
            start = time.time()
            print "epoch: ", epoch,", time spent: %.2fs" % time_lapse
            
        # output generated chairs
        if epoch % 500 == 0:
            g_images = sess.run(net_g_test.outputs,feed_dict={z_vector:z_sample}) # get a generated chair, with range [-1, 1]
            # make a directory for generated chairs
            if not os.path.exists(train_sample_directory):
                os.makedirs(train_sample_directory)
            
            #Save sample generated chair arrays
            g_images.dump(train_sample_directory+'/'+str(epoch))
        
        # save model check point
        if epoch % 500 == 0:
            # make a directory for trained models
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            
            # save the trained model at different epoch
            saver.save(sess, save_path = model_directory + '/' + str(epoch) + '.cptk')
    print "Done"

### Continue training from the last checkpoint

In [ ]:
saver = tf.train.Saver(max_to_keep=50)

In [ ]:
ckpt = tf.train.get_checkpoint_state(model_directory)
ckpt.all_model_checkpoint_paths

In [ ]:
ckpt.all_model_checkpoint_paths[3]

In [ ]:
print 'Loading models...might take a minute'
saver = tf.train.Saver(max_to_keep=50)

# create a log folder and save the graph structure, do this before training
g_writer = tf.train.SummaryWriter(logs_path + '/generator', graph=tf.get_default_graph())
d_writer = tf.train.SummaryWriter(logs_path + '/discriminator')

with tf.Session() as sess:  
    # set GPU memeory fraction
    tl.ops.set_gpu_fraction(sess=sess, gpu_fraction=0.99)
    
    ckpt = tf.train.get_checkpoint_state(model_directory)
    #model = ckpt.all_model_checkpoint_paths[3]
    model = ckpt.model_checkpoint_path
    saver.restore(sess, save_path=model)
    
    # -------- jointly training discriminator and generator --------
    start = time.time()
    
    # z noise vector that will be used to generate chairs to check the training progress
    #z_sample = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) # uniform distribution between [-1, 1]
    z_sample = np.random.normal(0, 0.33, size=[batch_size,z_size]).astype(np.float32) # gaussian distribution between [-1, 1]
    
    for epoch in range(4000, 25001):
        # get a batch of real chairs, with range [-1 ,1]
        x = get_x(batch_size) 
        # mini-batch of noise data from [-1, 1]
        #z = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32)
        z = np.random.normal(0, 0.33, size=[batch_size,z_size]).astype(np.float32)
    
        # Update the discriminator and generator
        d_summary_merge = tf.merge_summary([summary_d_loss, summary_d_x_hist,summary_d_z_hist])
        summary_d,discriminator_loss = sess.run([d_summary_merge,d_loss],feed_dict={z_vector:z, x_vector:x})
        summary_g,genterator_loss = sess.run([summary_g_loss,g_loss],feed_dict={z_vector:z})  
        #print "epoch: ", epoch
        #print 'd_loss:',discriminator_loss
        #print 'g_loss:',genterator_loss
        
        # only update the discriminator when its loss is larger than 10%   !!!!! D may not get sufficiently trained if threshold is too high
        if discriminator_loss <= 4.6*0.1: 
            sess.run([optimizer_op_g],feed_dict={z_vector:z})
            #print "epoch: ",epoch,', d_loss:',discriminator_loss,'g_loss:',genterator_loss
        # only update the generator when its loss is larger than 10%       !!!!! G may not get sufficiently trained if threshold is too high
        elif genterator_loss <= 4.6*0.1:
            sess.run([optimizer_op_d],feed_dict={z_vector:z, x_vector:x})
            #print "epoch: ",epoch,', d_loss:',discriminator_loss,'g_loss:',genterator_loss
        else:
            sess.run([optimizer_op_d],feed_dict={z_vector:z, x_vector:x})
            sess.run([optimizer_op_g],feed_dict={z_vector:z})
            
        # add loss summary to tensorboard
        if epoch % 10 == 0:
            d_writer.add_summary(summary_d, epoch) 
            g_writer.add_summary(summary_g, epoch)
        
        # print training progress
        if epoch % 100 == 0:
            time_lapse = time.time()-start
            start = time.time()
            print "epoch: ", epoch,", time spent: %.2fs" % time_lapse    
        
        # output generated chairs
        if epoch % 500 == 0:
            g_images = sess.run(net_g_test.outputs,feed_dict={z_vector:z_sample}) # get a generated chair, with range [-1, 1]
            # make a directory for generated chairs
            if not os.path.exists(train_sample_directory):
                os.makedirs(train_sample_directory)
            
            #Save sample generated chair arrays
            g_images.dump(train_sample_directory+'/'+str(epoch))
        
        # save model check point
        if epoch % 500 == 0:
            # make a directory for trained models
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            
            # save the trained model at different epoch
            saver.save(sess, save_path = model_directory + '/' + str(epoch) + '.cptk')
    print "Done"

### Turn a voxel 3d array into mesh, save as STL

In [ ]:
from stl import mesh
from skimage import measure
# Use marching cubes to obtain the surface mesh of these ellipsoids
vertices, faces = measure.marching_cubes(chair,0.0)

# Create the mesh and save as STL
chair = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        chair.vectors[i][j] = vertices[f[j],:]
# Write the mesh to STL file 
chair.save('matlab_chair.stl')